In [1]:
import pandas as pd
import os
from math import sqrt
import glob

In [2]:
def distancia_euclidiana(lat1, lon1, lat2, lon2):
    return sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2) * 111

In [ ]:
pasta_geocoded = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Processados"
pasta_finais = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Finais"
arquivo_estacoes = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Auxiliares\Locais de Coleta.xlsx"

In [4]:
os.makedirs(pasta_finais, exist_ok=True)

est_df = pd.read_excel(arquivo_estacoes)
RAIO_KM = 1.5

arquivos_geocoded = glob.glob(os.path.join(pasta_geocoded, "*_GEOCODED.*"))

In [5]:
if not arquivos_geocoded:
    print("Nenhum arquivo _GEOCODED encontrado!")
else:
    for caminho in arquivos_geocoded:
        arquivo = os.path.basename(caminho)
        # Lê dependendo da extensão
        if arquivo.lower().endswith(".xlsx"):
            df = pd.read_excel(caminho)
        elif arquivo.lower().endswith(".csv"):
            df = pd.read_csv(caminho, encoding='latin1', sep=';')
        else:
            print(f"Ignorado (extensão não suportada): {arquivo}")
            continue

        df = df.rename(columns=lambda x: x.strip().upper())

        # Verifica colunas necessárias: LATITUDE, LONGITUDE e LOCAL/ENDEREÇO
        if not ({"LATITUDE", "LONGITUDE"}.issubset(df.columns) and ("LOCAL" in df.columns or "ENDEREÇO" in df.columns)):
            print(f"Arquivo ignorado (colunas ausentes): {arquivo}")
            continue

        # Cria as colunas de resultado vazias
        df['Estação atribuída'] = None
        df['Distância (km)'] = None

        for idx, local in df.iterrows():
            lat_l = local['LATITUDE']
            lon_l = local['LONGITUDE']

            if pd.isna(lat_l) or pd.isna(lon_l):
                continue

            distancias = [(est['Estação'], distancia_euclidiana(lat_l, lon_l, est['LAT'], est['LONG']))
                        for _, est in est_df.iterrows()]

            dentro_raio = [(nome, d) for nome, d in distancias if d <= RAIO_KM]

            if len(dentro_raio) == 0:
                estacao_final = "sem estação correspondente"
                dist_final = None
            elif len(dentro_raio) == 1:
                estacao_final = dentro_raio[0][0]
                dist_final = dentro_raio[0][1]
            else:
                estacao_final, dist_final = min(dentro_raio, key=lambda x: x[1])

            # fallback LOCAL -> ENDEREÇO apenas para determinar o nome do local
            nome_local = local['LOCAL'] if 'LOCAL' in local and pd.notna(local['LOCAL']) else local.get('ENDEREÇO', None)

            # atribui os resultados nas novas colunas
            df.at[idx, 'Estação atribuída'] = estacao_final
            df.at[idx, 'Distância (km)'] = None if dist_final is None else round(dist_final, 4)

        # Nome do arquivo final
        nome_final = os.path.splitext(arquivo)[0] + "_FINAL.xlsx"
        caminho_saida = os.path.join(pasta_finais, nome_final)
        df.to_excel(caminho_saida, index=False)


Arquivo ignorado (colunas ausentes): BOMBEIROS_GEOCODED.csv
